In [2]:
# import pandas as pd
# import ray
# from ray.data.datasource import FastFileMetadataProvider

In [3]:
# lj_df = pd.read_csv(
#     "https://uberduck-datasets-dirty.s3.us-west-2.amazonaws.com/lj_for_upload/metadata_formatted_100.txt",
#     sep="|",
#     header=None,
#     quoting=3,
#     names=["path", "transcript", "speaker_id"], # pitch path is implicit - this should be changed
# )

# paths = lj_df.path.tolist()
# transcripts = lj_df.transcript.tolist()
# speaker_ids = lj_df.speaker_id.tolist()

# parallelism_length = 400
# audio_ds = ray.data.read_binary_files(
#     paths,
#     parallelism=parallelism_length,
#     meta_provider=FastFileMetadataProvider(),
#     ray_remote_args={"num_cpus": 0.2},
# )
# audio_ds = audio_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )

# paths = ray.data.from_items(paths, parallelism=parallelism_length)
# paths_ds = paths.map_batches(lambda x: x, batch_format="pyarrow", batch_size=None)

# transcripts = ray.data.from_items(transcripts, parallelism=parallelism_length)
# transcripts_ds = transcripts.map_batches(lambda x: x, batch_format="pyarrow", batch_size=None)

# speaker_ids_ds = ray.data.from_items(speaker_ids, parallelism=parallelism_length)
# speaker_ids_ds = speaker_ids_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# output_dataset = (
#     transcripts_ds.zip(audio_ds)
#     .zip(paths_ds)
#     .zip(speaker_ids_ds)
# )
# output_dataset = output_dataset.map_batches(
#     lambda table: table.rename(
#         columns={
#             "value_1": "transcript",
#             "value_2": "audio_bytes",
#             "value_3": "path",
#             "value_4": "speaker_id"
#         }
#     )
# )

In [1]:
from uberduck_ml_dev.exec.train_radtts_with_ray import get_ray_dataset, train_func
from ray.train.torch import TorchTrainer, TorchCheckpoint, TorchTrainer
class TorchCheckpointFixed(TorchCheckpoint):
    def __setstate__(self, state: dict):
        if "_data_dict" in state and state["_data_dict"]:
            state = state.copy()
            state["_data_dict"] = self._decode_data_dict(state["_data_dict"])
        super(TorchCheckpoint, self).__setstate__(state)



In [2]:
ray_dataset = get_ray_dataset()

2023-03-06 17:01:02,368	INFO worker.py:1360 -- Connecting to existing Ray cluster at address: 10.0.37.211:6379...
2023-03-06 17:01:02,376	INFO worker.py:1548 -- Connected to Ray cluster. View the dashboard at https://console.anyscale.com/api/v2/sessions/ses_ha6my3r3kl4mn9jdylcqnfutq1/services?redirect_to=dashboard 
2023-03-06 17:01:02,400	INFO packaging.py:330 -- Pushing file package 'gcs://_ray_pkg_ad1843ca6e36835e76ab0ad145e0012f.zip' (7.10MiB) to Ray cluster...
2023-03-06 17:01:02,518	INFO packaging.py:343 -- Successfully pushed file package 'gcs://_ray_pkg_ad1843ca6e36835e76ab0ad145e0012f.zip'.
(_get_read_tasks pid=321, ip=10.0.42.179) 2023-03-06 17:01:04,405	WARNING file_meta_provider.py:162 -- Expanding 100 path(s). This may be a HIGH LATENCY operation on some cloud storage services. If the specified paths all point to files and never directories, try rerunning this read with `meta_provider=FastFileMetadataProvider()`.


In [8]:
datum = ray_dataset.take(1)

(autoscaler +8m59s) Removing 1 nodes of type worker-node-type-1 (idle).


In [10]:
# datum

(autoscaler +9m23s) Removing 3 nodes of type worker-node-type-1 (idle).
(autoscaler +9m28s) Removing 3 nodes of type worker-node-type-1 (idle).
(autoscaler +9m34s) Removing 3 nodes of type worker-node-type-1 (idle).
(autoscaler +9m39s) Removing 3 nodes of type worker-node-type-1 (idle).
(autoscaler +9m44s) Removing 3 nodes of type worker-node-type-1 (idle).
(autoscaler +9m49s) Removing 3 nodes of type worker-node-type-1 (idle).
(autoscaler +9m54s) Removing 3 nodes of type worker-node-type-1 (idle).
(autoscaler +9m59s) Removing 3 nodes of type worker-node-type-1 (idle).
(autoscaler +10m4s) Removing 3 nodes of type worker-node-type-1 (idle).
(autoscaler +10m9s) Removing 3 nodes of type worker-node-type-1 (idle).
(autoscaler +10m14s) Removing 2 nodes of type worker-node-type-1 (idle).
(autoscaler +10m19s) Resized to 156 CPUs, 3 GPUs.
(autoscaler +10m19s) Removing 2 nodes of type worker-node-type-1 (idle).
(autoscaler +10m24s) Removing 2 nodes of type worker-node-type-1 (idle).
(autoscaler

In [4]:
from ray.air.config import ScalingConfig, RunConfig
from ray.tune import SyncConfig

In [5]:
from uberduck_ml_dev.exec.train_radtts_with_ray import train_config, model_config
train_config['n_group_size'] = model_config['n_group_size']
train_config['dur_model_config'] = model_config['dur_model_config']
train_config['f0_model_config'] = model_config['f0_model_config']
train_config['energy_model_config'] = model_config['energy_model_config']
train_config['v_model_config']=model_config['v_model_config']

In [6]:
trainer = TorchTrainer(
    train_loop_per_worker=train_func,
    train_loop_config=train_config,
    scaling_config=ScalingConfig(
        num_workers=2, use_gpu=True, resources_per_worker=dict(CPU=4, GPU=1)
    ),
    run_config=RunConfig(
        sync_config=SyncConfig(upload_dir="s3://uberduck-anyscale-data/checkpoints")
    ),
    datasets={"train": ray_dataset},
)

In [7]:
result = trainer.fit()

(autoscaler +1m31s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +1m31s) Adding 1 node(s) of type worker-node-type-1.


2023-03-06 17:03:33,074	WARNING insufficient_resources_manager.py:128 -- Ignore this message if the cluster is autoscaling. You asked for 9.0 cpu and 2.0 gpu per trial, but the cluster only has 84.0 cpu and 1.0 gpu. Stop the tuning job and adjust the resources requested per trial (possibly via `resources_per_trial` or via `num_workers` for rllib) and/or add more resources to your Ray runtime.
(raylet, ip=10.0.10.38) /home/ray/anaconda3/lib/python3.10/site-packages/ray/dashboard/agent.py:51: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.10.38)   aiogrpc.init_grpc_aio()


(autoscaler +2m47s) Resized to 152 CPUs, 2 GPUs.
(autoscaler +2m52s) Adding 2 node(s) of type worker-node-type-1.


(TrainTrainable pid=1713, ip=10.0.9.152) [nltk_data] Downloading package averaged_perceptron_tagger to
(TrainTrainable pid=1713, ip=10.0.9.152) [nltk_data]     /home/ray/nltk_data...
(TrainTrainable pid=1713, ip=10.0.9.152) [nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
(RayTrainWorker pid=192, ip=10.0.10.38) 2023-03-06 17:03:59,920	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=2]
(TorchTrainer pid=1713, ip=10.0.9.152) 2023-03-06 17:04:00,554	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[randomize_block_order]
(TorchTrainer pid=1713, ip=10.0.9.152) /home/ray/anaconda3/lib/python3.10/site-packages/ray/train/_internal/dataset_iterator.py:64: UserWarning: session.get_dataset_shard returns a ray.data.DatasetIterator instead of a Dataset/DatasetPipeline as of Ray v2.3. Use iter_torch_batches(), to_tf(), or iter_batches() to iterate over one epoch. See https://docs.ray.io/en/latest/data/api/dataset_iterat

(RayTrainWorker pid=192, ip=10.0.10.38) CUDA AVAILABLE:  True
(RayTrainWorker pid=192, ip=10.0.10.38) Applying spectral norm to text encoder LSTM
(RayTrainWorker pid=192, ip=10.0.10.38) Applying spectral norm to context encoder LSTM


...

Trial name,date,hostname,node_ip,pid,timestamp,trial_id
TorchTrainer_b7d1e_00000,2023-03-06_17-03-56,ip-10-0-9-152,10.0.9.152,1713,1678151036,b7d1e_00000


2023-03-06 17:04:35,823	ERROR tune.py:821 -- Trials did not complete: [TorchTrainer_b7d1e_00000]
2023-03-06 17:04:35,824	INFO tune.py:825 -- Total run time: 132.97 seconds (129.87 seconds for the tuning loop).


RayTaskError(TypeError): [36mray::_Inner.train()[39m (pid=1713, ip=10.0.9.152, repr=TorchTrainer)
  File "/home/ray/anaconda3/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 360, in train
    raise skipped from exception_cause(skipped)
  File "/home/ray/anaconda3/lib/python3.10/site-packages/ray/train/_internal/utils.py", line 54, in check_for_failure
    ray.get(object_ref)
ray.exceptions.RayTaskError(TypeError): [36mray::RayTrainWorker._RayTrainWorker__execute()[39m (pid=192, ip=10.0.10.38, repr=<ray.train._internal.worker_group.RayTrainWorker object at 0x7ff4ae83eec0>)
  File "/home/ray/anaconda3/lib/python3.10/site-packages/ray/train/_internal/worker_group.py", line 31, in __execute
    raise skipped from exception_cause(skipped)
  File "/home/ray/anaconda3/lib/python3.10/site-packages/ray/train/_internal/utils.py", line 129, in discard_return_wrapper
    train_func(*args, **kwargs)
  File "/tmp/ray/session_2023-03-06_08-59-42_925397_141/runtime_resources/wo

(raylet, ip=10.0.50.166) /home/ray/anaconda3/lib/python3.10/site-packages/ray/dashboard/agent.py:51: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.50.166)   aiogrpc.init_grpc_aio()
(raylet, ip=10.0.53.1) /home/ray/anaconda3/lib/python3.10/site-packages/ray/dashboard/agent.py:51: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.53.1)   aiogrpc.init_grpc_aio()


(autoscaler +4m3s) Resized to 160 CPUs, 4 GPUs.
(autoscaler +8m33s) Removing 1 nodes of type worker-node-type-1 (idle).
(autoscaler +8m38s) Removing 1 nodes of type worker-node-type-1 (idle).
(autoscaler +8m43s) Removing 1 nodes of type worker-node-type-1 (idle).
(autoscaler +8m48s) Removing 1 nodes of type worker-node-type-1 (idle).


In [2]:
import pandas as pd
import ray
parallelism_length = 400

In [3]:
lj_df = pd.read_csv(
    "https://uberduck-datasets-dirty.s3.us-west-2.amazonaws.com/lj_for_upload/metadata_formatted_100_edited.txt",
    sep="|",
    header=None,
    quoting=3,
    names=["path", "transcript", "speaker_id"], # pitch path is implicit - this should be changed
)
speaker_ids = lj_df.speaker_id.tolist()
speaker_ids_ds = ray.data.from_items(speaker_ids, parallelism=parallelism_length)
speaker_ids_ds = speaker_ids_ds.map_batches(
    lambda x: x, batch_format="pyarrow", batch_size=None
)

2023-03-06 16:54:26,264	INFO worker.py:1360 -- Connecting to existing Ray cluster at address: 10.0.37.211:6379...
2023-03-06 16:54:26,291	INFO worker.py:1548 -- Connected to Ray cluster. View the dashboard at https://console.anyscale.com/api/v2/sessions/ses_ha6my3r3kl4mn9jdylcqnfutq1/services?redirect_to=dashboard 
2023-03-06 16:54:26,316	INFO packaging.py:330 -- Pushing file package 'gcs://_ray_pkg_c450220a9fe1382fee31ab05c616a7cf.zip' (6.75MiB) to Ray cluster...
2023-03-06 16:54:26,438	INFO packaging.py:343 -- Successfully pushed file package 'gcs://_ray_pkg_c450220a9fe1382fee31ab05c616a7cf.zip'.


In [4]:
lj_df

,path,transcript,speaker_id
0,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,"Printing, in the only sense with which we are ...",0
1,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,in being comparatively modern.,0
2,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,For although the Chinese took impressions from...,0
3,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,"produced the block books, which were the immed...",0
4,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,the invention of movable metal letters in the ...,0
...,...,...,...
95,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,have now come into general use and are obvious...,0
96,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,a little reduced in ugliness. The design of th...,0
97,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,"and the whole effect is a little too gray, owi...",0
98,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,"It must be remembered, however, that most mode...",0


In [3]:
output_dataset = speaker_ids_ds.map_batches(
    lambda table: table.rename(
        columns={

            "value_1": "speaker_id"
        }
    )
)

In [4]:
output_dataset.speaker_id.tolist()

AttributeError: 'Dataset' object has no attribute 'speaker_id'

(autoscaler +2s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +2s) Removing 2 nodes of type worker-node-type-0 (idle).
(autoscaler +7s) Removing 2 nodes of type worker-node-type-0 (idle).
(autoscaler +12s) Removing 2 nodes of type worker-node-type-0 (idle).


In [ ]:
pd.DataFrame(output_dataset)

In [ ]:
output_dataset

2023-03-06 16:50:22,749	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(<lambda>)->MapBatches(<lambda>)]
MapBatches(<lambda>)->MapBatches(<lambda>): 100%|██████████| 100/100 [00:03<00:00, 28.88it/s]


In [39]:
#     lj_df = pd.read_csv(
#         "https://uberduck-datasets-dirty.s3.us-west-2.amazonaws.com/vctk_mic1/all_with_embs.txt",
#         sep="|",
#         header=None,
#         quoting=3,
#         names=["path", "speaker_id", "transcript", "dataset_audio_file_id", "emb_path"],
#     )

In [38]:
# lj_df['path'][0]

In [37]:
# transcripts_ds = ray.data.from_items(transcripts, parallelism=parallelism_length)
# speaker_ids_ds = ray.data.from_items(speaker_ids, parallelism=parallelism_length)
# dataset_audio_file_ids = ray.data.from_items(
#     dataset_audio_files, parallelism=parallelism_length
# )

# pitch_paths_ds = ray.data.read_binary_files(
#     pitch_paths,
#     parallelism=parallelism_length,
#     meta_provider=FastFileMetadataProvider(),
#     ray_remote_args={"num_cpus": 0.2},
# )

# audio_ds = audio_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# transcripts_ds = transcripts_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# dataset_audio_file_ids = dataset_audio_file_ids.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# paths_ds = paths.map_batches(lambda x: x, batch_format="pyarrow", batch_size=None)
# pitches_paths_ds = pitch_paths_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# speaker_ids_ds = speaker_ids_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
